In [1]:
from sklearn.linear_model import LinearRegression
import pandas as pd


In [5]:
train = pd.read_csv("train.csv")
test = pd.read_csv("test.csv")
print(train.head(5))

   label  pixel0  pixel1  pixel2  pixel3  pixel4  pixel5  pixel6  pixel7  \
0      1       0       0       0       0       0       0       0       0   
1      0       0       0       0       0       0       0       0       0   
2      1       0       0       0       0       0       0       0       0   
3      4       0       0       0       0       0       0       0       0   
4      0       0       0       0       0       0       0       0       0   

   pixel8    ...     pixel774  pixel775  pixel776  pixel777  pixel778  \
0       0    ...            0         0         0         0         0   
1       0    ...            0         0         0         0         0   
2       0    ...            0         0         0         0         0   
3       0    ...            0         0         0         0         0   
4       0    ...            0         0         0         0         0   

   pixel779  pixel780  pixel781  pixel782  pixel783  
0         0         0         0         0         

In [38]:
predictors = train.columns.tolist()
predictors.remove('label')


In [27]:
from sklearn import cross_validation
from sklearn import neighbors
predictors = train.columns
knn = neighbors.KNeighborsClassifier(n_neighbors=5)
scores = cross_validation.cross_val_score(knn, train[predictors], train["label"], cv=3)


<function mean>

In [29]:
scores.mean()

0.9642142332407303

In [39]:
knn.fit(train[predictors], train["label"])

KNeighborsClassifier(algorithm='auto', leaf_size=30, metric='minkowski',
           metric_params=None, n_jobs=1, n_neighbors=5, p=2,
           weights='uniform')

In [33]:
test.head(5)
test.shape

(28000, 784)

In [40]:
predicted = knn.predict(test)


In [47]:
id = list(range(1,len(test)+1))


In [49]:
submission = pd.DataFrame({
        "ImageId": id,
        "label": predicted
    })

In [50]:
submission.to_csv("output.csv")

In [57]:
from sklearn.ensemble import RandomForestClassifier


In [58]:
alg = RandomForestClassifier(random_state=1, n_estimators=150, min_samples_split=4, min_samples_leaf=2)

In [59]:
scores = cross_validation.cross_val_score(alg,train[predictors],train["label"],cv=3)

In [60]:
scores.mean()

0.96195260725825316

In [106]:
import numpy as np
from sklearn.feature_selection import SelectKBest, f_classif
selector = SelectKBest(f_classif, k=300)
selector.fit(train[predictors], train["label"])

SelectKBest(k=300, score_func=<function f_classif at 0x7f95080a3410>)

In [107]:
mask = selector.get_support(True)
mask
newPredictors = list( predictors[i] for i in mask )


In [105]:
knn = neighbors.KNeighborsClassifier(n_neighbors=5)
scoresNew = cross_validation.cross_val_score(knn, train[newPredictors], train["label"], cv=3)
scoresNew.mean()

0.9642380869768844

In [108]:
knn.fit(train[newPredictors], train["label"])


ValueError: query data dimension must match training data dimension

In [109]:
predictedNew = knn.predict(test[newPredictors])
submission = pd.DataFrame({
        "ImageId": id,
        "label": predictedNew
    })
submission.to_csv("output1.csv")

In [110]:
from sklearn import svm


In [111]:
clf = svm.SVC(kernel='poly', C=.01, degree=3)
scoresSVC = cross_validation.cross_val_score(clf, train[newPredictors], train["label"], cv=3)
scoresSVC.mean()

0.96980946476256324

In [112]:
clf.fit(train[newPredictors], train["label"])
predictedSVC = clf.predict(test[newPredictors])



In [113]:
submission = pd.DataFrame({
        "ImageId": id,
        "label": predictedSVC
    })
submission.to_csv("output2.csv")